# Introduction

#### This etl project collects and presents user activity information from a streaming service called Sparkify

# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/jabulaninyoni/Documents/udacity/data_engineering/project_2_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating tables and inserting data into Apache Cassandra

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
#Keyspace created
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Below three queries will answer the questions put forward


## Query 1

this query we want to return artist, song_title and length. We can assume that *session_id* will always be unique. This means that we can partition on this since records of the same session will be stored together. Secondly the query also wants us to sort by *item_in_session* we will add this as an extra clustering key.



In [10]:
## Creating a table that gives the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = """
        CREATE TABLE IF NOT EXISTS artist_song_length(
        session_id int,
        item_in_session int,
        artist_name text,
        song_title text,
        song_length double,
        PRIMARY KEY (session_id, item_in_session)
        )
        """
try:
    session.execute(query)
except Exception as e:
     print(e)

                    

#### Insert data into table

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_length(session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist, song, float(length)))

#### Query table to check data in table is accurate

In [12]:
query = """
        select artist_name, song_title, song_length from artist_song_length where session_id = 338 
        and item_in_session = 4
        """
try:
    result = session.execute(query)
    for row in result:
        print(row.artist_name, row.song_title, row.song_length)
except Exception as e:
     print(e)


Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2

#####  In this query we want to return name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 


##### To achieve this we are going to use a composite partition key for (user_id and session_id) and a clustering key for item_in_session. We are using a composite paritition key because we want to filter by both (user_id and session_id ). In addition to this we also want to sort by itemInSession that is why we have it as a clustering column.
    



In [13]:
## Create a table with name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = """
        CREATE TABLE IF NOT EXISTS artist_song_user(
        user_id int,
        session_id int,
        item_in_session int,
        artist_name text,
        song_title text,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY ((user_id, session_id), item_in_session)
        )
        """
try:
    session.execute(query)
except Exception as e:
     print(e)

                    

#### Insert data into table

In [14]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user(user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId),int(sessionId), int(itemInSession), artist, song, firstName, lastName))

#### Query table to check data in table is accurate

In [15]:
# i added item_in_session in the output so i can see that it is sorted by that column
query = """
        select item_in_session, artist_name, song_title, user_first_name, user_last_name from artist_song_user where user_id = 10 
        and session_id = 182
        """
try:
    result = session.execute(query)
    for row in result:
        print(row.item_in_session, row.artist_name, row.song_title, row.user_first_name, row.user_last_name)
except Exception as e:
     print(e)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3

#####  In this query we want to return user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' 


##### To achieve this we are going to use song_title as our partition key since we are filtering on it and the user_id as the clustering key. The assumption is that for each user_id is unique to each user i.e. user first and last name
 


In [16]:
## Create a table for every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = """
        CREATE TABLE IF NOT EXISTS user_by_song(
        song_title text,
        user_id int,
        user_first_name text,
        user_last_name text,
        artist_name text,
        song_length double,
        PRIMARY KEY (song_title, user_id)
        )
        """
try:
    session.execute(query)
except Exception as e:
     print(e)

                    

#### Insert data into table

In [17]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song(song_title, user_id, user_first_name, user_last_name, artist_name, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, int(userId),firstName, lastName, artist, float(length)))

#### Query table to check data in table is accurate

In [18]:
# every user name (first and last)
query = """
        select user_first_name, user_last_name from user_by_song
        where song_title = 'All Hands Against His Own' 
        """
try:
    result = session.execute(query)
    for row in result:
        print(row.user_first_name, row.user_last_name)
except Exception as e:
     print(e)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
query = """
        DROP TABLE IF EXISTS artist_song_length
        """
try:
    session.execute(query)
except Exception as e:
     print(e)
        
query = """
        DROP TABLE IF EXISTS artist_song_user
        """
try:
    session.execute(query)
except Exception as e:
     print(e)
        
query = """
        DROP TABLE IF EXISTS user_by_song
        """
try:
    session.execute(query)
except Exception as e:
     print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()